Dataset com base https://www.kaggle.com/thedownhill/art-images-drawings-painting-sculpture-engraving/home

Context: 
Dataset for classifying different styles of art. Main categories have been taken here<br>
Content: 
5 kinds of data downloaded from google images, yandex images and this site:

Drawings and watercolours<br>
Works of painting<br>
Sculpture<br>
Graphic Art<br>
Iconography (old Russian art)<br>


Data is separated on training and validation sets

In [31]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
import os, shutil
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from collections import Counter
import keras

In [32]:
base_dir = './dataset'

train_dir = os.path.join(base_dir, 'training_set')
validation_dir = os.path.join(base_dir, 'validation_set')

In [34]:
# redimensionando as imagens 

from PIL import Image

os.mkdir(os.path.join(base_dir, 'training_set_convert'))
os.mkdir(os.path.join(base_dir, 'validation_set_convert'))
train_dir_convert = os.path.join(base_dir, 'training_set_convert')
validation_dir_convert = os.path.join(base_dir, 'validation_set_convert')
for i in labels:
    os.mkdir(os.path.join(train_dir_convert, i))
    os.mkdir(os.path.join(validation_dir_convert, i))

for i in labels:
    _dir = os.path.join(train_dir, i)
    _dir_conv = os.path.join(train_dir_convert, i)
    for x in os.listdir(_dir):
        try:
            img = Image.open(os.path.join(_dir, x))
            img = img.resize((28,28), Image.ANTIALIAS)
            img.save(os.path.join(_dir_conv, x))
        except Exception:
            pass


In [35]:
# adicionar rotulos as imagens.

train_dir = train_dir_convert
validation_dir_convert = validation_dir_convert

# criando os rotulos
labels = [x for x in os.listdir(train_dir)]
print('labels -> %s\n' % labels)

# rotulo por image
y_train = []
y_test = []

print('No Conjunto de Treino:')
for i in labels:
    _dir = os.path.join(train_dir, i)
    print('Label -> %s, Tamanho -> %s' % (i, len(os.listdir(_dir))))
    
print('\nNo Conjunto de Teste:')
for i in labels:
    _dir = os.path.join(validation_dir, i)
    print('Label -> %s, Tamanho -> %s' % (i, len(os.listdir(_dir))))

for i in labels:
    _dir = os.path.join(train_dir, i)
    for x in range(len(os.listdir(_dir))):
        y_train.append(i)
    
for i in labels:
    _dir = os.path.join(validation_dir, i)
    for x in range(len(os.listdir(_dir))):
        y_test.append(i)


def frequecia(lista):
    d = Counter(lista)
    n, m = d.keys(), d.values()
    print([i for i in zip(n,m)])  

print('')
frequecia(y_test)
frequecia(y_train)

labels -> ['drawings', 'engraving', 'iconography', 'painting', 'sculpture']

No Conjunto de Treino:
Label -> drawings, Tamanho -> 1107
Label -> engraving, Tamanho -> 757
Label -> iconography, Tamanho -> 2077
Label -> painting, Tamanho -> 2042
Label -> sculpture, Tamanho -> 1738

No Conjunto de Teste:
Label -> drawings, Tamanho -> 122
Label -> engraving, Tamanho -> 84
Label -> iconography, Tamanho -> 231
Label -> painting, Tamanho -> 236
Label -> sculpture, Tamanho -> 193

[('painting', 236), ('sculpture', 193), ('engraving', 84), ('drawings', 122), ('iconography', 231)]
[('painting', 2042), ('sculpture', 1738), ('engraving', 757), ('drawings', 1107), ('iconography', 2077)]


In [48]:
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir, 
                                                    target_size=(28,28), 
                                                    batch_size=20, 
                                                    class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(validation_dir, 
                                                              target_size=(28,28), 
                                                              batch_size=20, 
                                                              class_mode='categorical')

def generate_data_generator_for_two_images(X1, X2, Y):
    genX1 = train_datagen.flow(X1,Y, seed=7)
    genX2 = train_datagen.flow(X2, seed=7)
    while True:
            X1i = genX1.next()
            X2i = genX2 .next()
            yield [X1i[0], X2i ], X1i[1]

train_datagen = generate_data_generator_for_two_images(28,28,3)
            

Found 7721 images belonging to 5 classes.
Found 866 images belonging to 5 classes.


In [49]:
model = Sequential()
model.add(Dense(32, activation='relu', input_shape=(28,28,3)))
model.add(Dense(3, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 28, 28, 32)        128       
_________________________________________________________________
dense_16 (Dense)             (None, 28, 28, 3)         99        
Total params: 227
Trainable params: 227
Non-trainable params: 0
_________________________________________________________________


In [50]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit_generator(
    train_generator,
    steps_per_epoch=7820/20, 
    epochs=10, 
    validation_data=validation_generator, 
    validation_steps=866/20)

Epoch 1/10


ValueError: Error when checking target: expected dense_16 to have 4 dimensions, but got array with shape (20, 5)